In [2]:
# Turnstile Data

import numpy as np
import pandas as pd

import datetime as dt
import time

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy import stats
from scipy.stats import gamma

import geopandas as gp
from geopandas import sjoin
import shapely as shp
from shapely.geometry import Point
import fiona as fio
from fiona.crs import from_epsg

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
mta_2016 = pd.read_csv('data/mta_2016.csv')

In [4]:
mta_2016.drop(['Unnamed: 0'],axis=1,inplace=True)
mta_2016['PLATFORM'] = mta_2016['STATION']+" - "+mta_2016['LINENAME']
mta_2016['DT'] = pd.to_datetime(mta_2016['DATE'])
mta_2016.rename(columns={'EXITS                                                               ':'EXITS'},inplace=True)
mta_2016 = mta_2016[mta_2016['DT'].dt.year == 2016]

In [5]:
df = mta_2016[mta_2016.DESC == 'REGULAR']
df = df.groupby(['C/A', 'UNIT', 'STATION','PLATFORM', 'SCP', 'DT']).sum()

In [6]:
df["ENTRIES_ACT"] = df.groupby(level=[0,1,2,3])['ENTRIES'].transform(lambda x: abs(x - x.shift()))
df["EXITS_ACT"] = df.groupby(level=[0,1,2,3])['EXITS'].transform(lambda x: abs(x - x.shift()))

In [7]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [8]:
df.drop(['ENTRIES', 'EXITS'], axis=1, inplace=True)
df.reset_index(inplace=True)

In [9]:
df.ENTRIES_ACT.quantile(0.9993)

2727.9642999991775

In [10]:
df.EXITS_ACT.quantile(0.9993)

3185.0

In [11]:
reduced_entries = df[df.ENTRIES_ACT < df.ENTRIES_ACT.quantile(0.9993)]['ENTRIES_ACT']
    
lower_thresh_entries = 0
upper_thresh_entries = reduced_entries.mean() + 3 * reduced_entries.std()
                                                                           

reduced_exits = df[df.EXITS_ACT < df.EXITS_ACT.quantile(0.9993)]['EXITS_ACT']
    
lower_thresh_exits = 0
upper_thresh_exits = reduced_exits.mean() + 3 * reduced_exits.std()
 

In [12]:
print (reduced_entries.mean(),upper_thresh_entries)
print (reduced_exits.mean(),upper_thresh_exits)


(182.74373632809838, 975.603420372128)
(138.05732175457553, 832.9042776247456)


In [13]:
outlier_mask_entries = df['ENTRIES_ACT'].map(lambda x: lower_thresh_entries <= x <= upper_thresh_entries)
    
outlier_mask_exits = df['EXITS_ACT'].map(lambda x: lower_thresh_exits <= x <= upper_thresh_exits)
    
df = df[outlier_mask_entries & outlier_mask_exits]

In [14]:
df = df.set_index('DT').groupby([pd.TimeGrouper('H'), 'PLATFORM']).sum()

In [15]:
df.reset_index(inplace=True)

In [16]:
df.head(5)

,DT,PLATFORM,index,ENTRIES_ACT,EXITS_ACT
0,2016-01-01,23 ST - 6,7544606,478.0,96.0
1,2016-01-01,B'WAY-LAFAYETTE - BDFQ6,5074266,281.0,163.0
2,2016-01-01,BUHRE AV - 6,8850089,293.0,314.0
3,2016-01-01,CANAL ST - JNQRZ6,386264,598.0,321.0
4,2016-01-01,CORTLANDT ST - R,491348,823.0,341.0


In [17]:
date_range = pd.date_range(start='2016-01-01', end='2016-12-31')

cal = calendar()
holidays = cal.holidays(start=date_range.min(), end=date_range.max())

In [18]:
df_7_11 = df[(df.DT.dt.hour >= 7) & (df.DT.dt.hour <= 11)] #.groupby('PLATFORM').count()
df_7_11_wkdy = df_7_11[(df_7_11.DT.dt.weekday >= 0) & (df_7_11.DT.dt.weekday <= 4)]
df_workdays = df_7_11_wkdy[df_7_11_wkdy.DT.dt.date.astype('datetime64').isin(holidays) != True]


In [19]:
df_workdays_count_by_day = df_workdays.set_index('DT').groupby([pd.TimeGrouper('D'), 'PLATFORM']
                                   ).count().reset_index().sort_values(by=['PLATFORM','DT'])

In [20]:
df_workdays_count = df_workdays.set_index('DT').groupby('PLATFORM'
                                   ).count().reset_index().sort_values(by='PLATFORM')

In [21]:
df_workdays_sum_by_day = df_workdays.set_index('DT').groupby([pd.TimeGrouper('D'), 'PLATFORM']
                                   ).sum().reset_index().sort_values(by=['PLATFORM','DT'])

In [22]:
df_morn_rush = df_workdays_sum_by_day.groupby('PLATFORM').mean().reset_index().sort_values(by='PLATFORM').drop('index',axis=1)

In [23]:
df_morn_rush.rename(columns={'ENTRIES_ACT':'Entries','EXITS_ACT':'Exits'},inplace=True)

In [24]:
df_morn_rush[df_morn_rush['PLATFORM'] == '42 ST-PORT AUTH - ACENGRS1237']

,PLATFORM,Entries,Exits
110,42 ST-PORT AUTH - ACENGRS1237,1286.941748,2408.524272


In [25]:
df_workdays_count.head(4)

,PLATFORM,index,ENTRIES_ACT,EXITS_ACT
0,1 AV - L,358,358,358
1,103 ST - 1,266,266,266
2,103 ST - 6,268,268,268
3,103 ST - BC,259,259,259


In [26]:
df_morn_rush.to_csv('data/mta_morn_rush.csv')

In [ ]:
### DIAGNOSTICS......

In [27]:
df_morn_rush.sort_values(by='Exits',ascending=False).head(10)

,PLATFORM,Entries,Exits
96,34 ST-HERALD SQ - BDFMNQRW,12464.809524,16646.619048
323,FULTON ST - 2345ACJZ,5468.908367,13702.924303
333,GRD CNTRL-42 ST - 4567S,10444.430279,13153.071713
95,34 ST-HERALD SQ - BDFMNQR,9117.645933,12090.387560
136,59 ST - 456NQRW,4848.738095,11877.119048
475,TIMES SQ-42 ST - 1237ACENQRSW,6952.833333,10172.642857
100,34 ST-PENN STA - ACE,13903.000000,9492.593625
117,47-50 STS ROCK - BDFM,702.107570,9046.446215
376,LEXINGTON AV/53 - EM6,1880.051793,8743.043825
474,TIMES SQ-42 ST - 1237ACENQRS,5335.924303,8301.366534


In [ ]:
#mta_2016[mta_2016['PLATFORM'] == stn].groupby(['PLATFORM','C/A','UNIT','SCP','DT']).max()

In [ ]:
#mta_2016.groupby(['STATION','LINENAME','C/A','UNIT','SCP']).max()

In [ ]:
#mta_2016.groupby(['C/A','UNIT'])[['C/A','UNIT']].max()

In [ ]:
#mta_2016.groupby('C/A').count()

In [ ]:
#mta_2016.groupby('UNIT').count()

In [ ]:
#mta_2016.groupby(['PLATFORM']).count().sort_values(by='EXITS',ascending=False)

In [ ]:
#mta_2016.sort_values(by=['DT']).head(5)
#mta_2016[(mta_2016['DT'].dt.month == 1) & (mta_2016['DT'].dt.day == 1) & (mta_2016['DT'].dt.hour == 0)].sort_values(by=['PLATFORM','DT'])

In [ ]:
#mta_2016[(mta_2016['DT'].dt.month == 1) & (mta_2016['DT'].dt.day == 1) & (mta_2016['DT'].dt.hour == 0)].sort_values(by=['PLATFORM','DT']).groupby('PLATFORM').agg(['min', 'max'])

In [ ]:
#mta_2016.groupby('DT').iloc[0:7,:]

#.DT.dt.date(2016,1,1)

In [ ]:
#mta_2016.groupby('PLATFORM')['ENTRIES'].max()
#mta_group = mta_2016[
#    ['STATION','PLATFORM','ENTRIES','EXITS','DT']][(mta_2016['DT'].dt.year == 2016) & (mta_2016['DT'].dt.dayofyear > 1)
#                                               ].sort_values(
#    by=['PLATFORM','DT']).groupby('PLATFORM')['STATION','ENTRIES','EXITS','DT'].agg(['min', 'max'])



In [ ]:
#mta_group_tot_doy = pd.DataFrame(mta_group['DT']['max'] - mta_group['DT']['min'])